In [1]:
import os
import torch
from torch import nn
import torchtext
from torchtext.data import get_tokenizer
import numpy as np
import pandas as pd
import json
from torch.utils.data import Dataset , DataLoader

import nltk 
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/wujh1123/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/wujh1123/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub(r"(?:\@)\S+", "", text)
    text = re.sub(r"(?:\#)\S+", "", text)
    pun = re.compile("[" u"\u2014" u"\u2019" u"\u00a3" u"\u20ac" "\u00b4" "]+",re.UNICODE)
    text = re.sub(pun, ' ', text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [3]:
def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats

        u"\u3030"
        u"\u201c"
        u"\u201d"
        u"\u2049"
        u"\ufe0f"
        u"\u2026" 
        u"\u203c"
        u"\u2018"
        u"\u2066"
        u"\u201e"
        u"\u200b"

                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)



In [4]:
stop_words = stopwords.words('english')
def remove_stopwords(text):
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

In [5]:
lemmatizer = WordNetLemmatizer()
def lem_text(text):
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split(' '))
    return text

In [6]:
def preprocess(text):
    text = clean_text(text)
    text = remove_emoji(text)
    text = remove_stopwords(text)
    text = lem_text(text)
    text = text.split()
    return text

In [7]:
s = "@4mishee @ScottMorrisonMP can keep his vaccine \ud83d\udc89 I\u2019ll take covid with its \u201c non existent \u201c symptoms over some big pharma no liability or indemnity lab \ud83e\uddea cooked vaccine thank you."
t = preprocess(s)
print(t)

['keep', 'vaccine', 'take', 'covid', 'non', 'existent', 'symptom', 'big', 'pharma', 'liability', 'indemnity', 'lab', 'cooked', 'vaccine', 'thank']


In [8]:
def load_data(path):
    with open(path,'r',encoding='utf-8') as file:
        data = json.load(file)
    # print(len(train_data))
    return data

In [9]:
train_data =load_data("train.json")
sent_arr = set(preprocess(train_data[0]['tweet']))
for key,value in train_data[0]['labels'].items():
    for i in value:
        print(i['terms'])
   

cant control the Flu
infectious diseases


In [10]:
train_data =load_data("train.json")
words = set()
for i in train_data:
    for k,v in i['labels'].items():
        sent_arr = set(preprocess(i['tweet']))
        if len(sent_arr) == 0:
            print(i['tweet'])
            train_data.remove(i)
            continue        
        for j in v:
            sent_arr = set(preprocess(j['terms']))
            words = words | sent_arr




words = ["","UNK"]+list(words)
print(len(words))

[The Oxford-AstraZeneca vaccine also seems to produce relatively high rates of adverse events. If you want to dig further into this vaccine’s story and issues, I’ve laid out a more detailed rundown of the Oxford-AstraZeneca trials and sources here.] https://t.co/youZFn8KUr
4500


In [11]:
# # import spacy
# # nlp = spacy.load('en_core_web_sm')
# # tokenizer = get_tokenizer('spacy')
# train_data =load_data("train.json")
# words = set()
# for i in train_data:
#     sent_arr = set(preprocess(i['tweet']))
#     if len(sent_arr) == 0:
#         print(i['tweet'])
#         train_data.remove(i)
#         continue
#     words = words | sent_arr



# words = ["","UNK"]+list(words)
# print(len(words))

In [12]:
# from collections import Counter
# train_data =load_data("train.json")
# counts = Counter()
# for i in train_data:
#     sent_arr = (preprocess(i['tweet']))
#     if len(sent_arr) == 0:
#         print(i['tweet'])
#         train_data.remove(i)
#     counts.update(sent_arr)


# print("num_words before:",len(counts.keys()))
# for word in list(counts):
#     if counts[word] < 2:
#         del counts[word]
# print("num_words after:",len(counts.keys()))
# words = ["", "UNK"]
# for word in counts:
#     words.append(word)

In [13]:
# from collections import Counter
# train_data =load_data("train.json")

# counts = Counter()
# for i in train_data:
#     for k,v in i['labels'].items():
#         sent_arr = set(preprocess(i['tweet']))
#         if len(sent_arr) == 0:
#             print(i['tweet'])
#             train_data.remove(i)
#             continue        
#         for j in v:
#             sent_arr = set(preprocess(j['terms']))
#             counts.update(sent_arr)




# print("num_words before:",len(counts.keys()))
# for word in list(counts):
#     if counts[word] < 2:
#         del counts[word]
# print("num_words after:",len(counts.keys()))
# words = ["", "UNK"]
# for word in counts:
#     words.append(word)

In [14]:
tag_list=["ineffective","unnecessary","pharma","rushed","side-effect","mandatory","country","ingredients","political","none","conspiracy","religious"]
tag_to_idx={}
for i in range(len(tag_list)):
    tag_to_idx[tag_list[i]]=i

In [15]:
print(type(train_data[0]['labels']))
print(train_data[0]['labels'])
print(list(train_data[0]['labels'].keys()))
for i in list(train_data[77]['labels'].keys()):
    print(tag_to_idx[i],i)

<class 'dict'>
{'ineffective': [{'index': 0, 'start': 6, 'end': 10, 'terms': 'cant control the Flu'}, {'index': 1, 'start': 37, 'end': 39, 'terms': 'infectious diseases'}]}
['ineffective']
4 side-effect


In [16]:
vocab2idx = {k:v for v,k in enumerate(words)}
vocab_size = len(words)
vocab_size

4500

In [17]:
def load_glove_vectors(glove_file="glove.42B.300d.txt"):
    word_vectors = {}
    with open(glove_file) as f:
        for line in f :
            s = line.split(' ')
            word_vectors[s[0]] = np.array([float(x) for x in s[1:]])
    return word_vectors

In [18]:
def get_emb_matrix(pretained,words,emb_size=300):
    W = np.zeros((len(words),emb_size),dtype="float32")
    W[0] = np.zeros(emb_size,dtype="float32")
    W[1] = np.random.uniform(-0.25,0.25,emb_size)
    i = 2
    count = 0 
    no_use = []
    for word in words[2:]:
        if word in pretained:
            W[i] = pretained[word]
            count +=1
        else:
            W[i] =  np.random.uniform(-0.25,0.25,emb_size)
            no_use.append(word)
        i+=1
    rate = count/len(words)
    print("coverage rate: " ,round(rate, 4))
    print(no_use)
    print(len(no_use))
    return W

In [19]:
# test for some specil unicode
punctuation = "×"

for char in punctuation:
    unicode_code = ord(char)
    print(f"Character: {char}, Unicode Code: {hex(unicode_code)}")

Character: ×, Unicode Code: 0xd7


In [20]:
import joblib
word_vecs = load_glove_vectors()
pretrain_weights = get_emb_matrix(word_vecs,words,300)
# joblib.dump(pretrain_weights, 'pretrain.pkl')

coverage rate:  0.976
['fauxi', 'superspreaders', 'drumps', 'manpfizer', 'umsuzwane', 'monehhh', 'yeaes', 'curevac', 'azanediyl', 'vaxx', 'govm', 'succussfully', 'humantissues', 'unaproved', 'covaxine', 'permanate', 'faucy', 'untrialled', 'covaxin', 'zoflot', 'recations', 'undertested', 'covid', 'fauchi', 'doncovfefe', 'valneva', 'remdesivir', 'eficacy', 'catastrophicaly', 'antiechrist', 'covishield', 'pharmedia', 'nanofilaments', 'agenca', 'peridcarditis', 'pyzer', 'assymptomatic', 'thundercunts', 'vaxxed', 'plandemic', 'goldpot', 'vaxxer', 'hexyldecanoate', 'ofadverse', 'astrazenica', 'unfully', 'inpfizer', 'hypercarbic', 'scamdemicgoal', 'preclinicals', 'modrna', 'brexit', 'vacclne', 'inferti', 'billygates', 'vacinnes', 'astrazeneka', 'veklury', 'vacunal', 'profitspeople', 'vaccitech', 'mneongreen', 'cvsts', 'eeiriely', 'biontec', 'personanymoreyou', 'coronavac', '×', 'stopworldcontroldotcom', 'extg', 'trudick', 'mnra', 'stabelizer', 'cormorabities', 'overflating', 'novichok', 'mand

In [21]:


class CustomDataset(Dataset):
    def __init__(self,data, tokenizer,max_length,words):
        self.data = data 
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.words = words

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index) :
        tweet = self.data[index]["tweet"]
        labels = self.data[index]["labels"]
        encode_text = self.encode(tweet,self.words)
        encode_label = self.label_encoded(labels)
        seq_len = np.count_nonzero(encode_text)
        # seq_len = len(encode_text)
        return encode_text , encode_label , seq_len
    def encode(self,text,vocab2idx):
        token = self.tokenizer(text)
        encoded = np.zeros(self.max_length,dtype=int)
        enc1 = np.array([vocab2idx.get(word,vocab2idx["UNK"]) for word in token] )
        # print(enc1.shape)
        length = min(self.max_length,len(enc1))
        encoded[:length] = enc1[:length]
        
        return encoded 
    def label_encoded(self,labels):
        encode_label = np.zeros(12,dtype=int)
        list_label = list(labels.keys())
        for i in list_label:
            idx = tag_to_idx[i]
            encode_label[idx] = 1
        # encode_label = torch.LongTensor(encode_label)
        return encode_label 

In [22]:
max_length = 300
train_data = CustomDataset(train_data,preprocess,max_length,vocab2idx)
train_loader = DataLoader(train_data,batch_size=32)


In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Now is using {device} device")

Now is using cuda device


In [24]:
from torch.nn.utils.rnn import pack_padded_sequence ,pad_packed_sequence

In [25]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch , (X,y,seq) in enumerate(dataloader):
        X,y =X.to(device),y.to(device,dtype=torch.float)
        # print(seq)
        pred = model(X,seq)
        # print(pred.dtype)
        loss = loss_fn(pred,y)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%200 ==0:
            loss,curr = loss.item() , batch * len(X)
            print(f"loss: {loss:>7f} [{curr:>5d}/{size:>5d}]")

In [208]:
import torch.nn.functional as F
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

class pretrain_weight_LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim,hidden_dim,pretrain_weights,bidirectional) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size,embedding_dim,padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrain_weights))
        self.embeddings.weight.requires_grad = True 
        self.lstm = nn.LSTM(embedding_dim,hidden_dim,num_layers=1,bidirectional=bidirectional,batch_first=True)
        self.embedding_dropout = SpatialDropout(0.5)
        self.relu = nn.ReLU()
        h = int(hidden_dim*3)
        self.tanh= nn.Tanh()
        self.fc1 = nn.Linear(in_features=hidden_dim*2,out_features=h)
        self.droupt1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(h,12)
        self.droupt2 = nn.Dropout(0.5)
        self.w_omega = nn.Parameter(torch.Tensor(hidden_dim*2,hidden_dim*2))
        self.u_omega = nn.Parameter(torch.Tensor(hidden_dim*2,1))
        self.word_bias = nn.Parameter(torch.Tensor(1, 2 * hidden_dim))
        # nn.init.uniform_(self.w_omega,-0.1,0.1)
        # nn.init.uniform_(self.u_omega,-0.1,0.1)
        nn.init.xavier_uniform_(self.w_omega)
        nn.init.xavier_uniform_(self.u_omega)

    def forward(self,x,s):
        embeed = self.embeddings(x)
        embeed = self.embedding_dropout(embeed)
        x_pack = pack_padded_sequence(embeed,s.to('cpu'),batch_first=True, enforce_sorted=False)
        
        pack_out , (ht,ct) = self.lstm(x_pack)
        # ht = torch.cat((ht[-2,:,:], ht[-1,:,:]), dim = 1)
        # out = self.droupt(ht[-1])
        x,length =pad_packed_sequence(pack_out, batch_first=True)
        # print(x.shape)
        u = torch.tanh(torch.matmul(x,self.w_omega)+self.word_bias)
        att = torch.matmul(u,self.u_omega)
        att_score = F.softmax(att,dim=1)
        score_x = x*att_score
        ht = torch.sum(score_x,dim=1)
        # ht = self.tanh(ht)
        out = self.droupt1(ht)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.droupt2(out)
        out = self.fc2(out)
        return out 
    

In [209]:
bidirectional = True
embedding_dim = 300
hidden_dim = 192
model_2 = pretrain_weight_LSTM(vocab_size,embedding_dim,hidden_dim,pretrain_weights,bidirectional).to(device)

In [210]:
from torchsummary import summary
model_2

pretrain_weight_LSTM(
  (embeddings): Embedding(4500, 300, padding_idx=0)
  (lstm): LSTM(300, 192, batch_first=True, bidirectional=True)
  (embedding_dropout): SpatialDropout(p=0.5, inplace=False)
  (relu): ReLU()
  (tanh): Tanh()
  (fc1): Linear(in_features=384, out_features=576, bias=True)
  (droupt1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=576, out_features=12, bias=True)
  (droupt2): Dropout(p=0.5, inplace=False)
)

In [211]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_2.parameters(),lr=1e-3)

In [212]:
from sklearn.metrics import f1_score

In [213]:
def evalute(dataloader,model,loss_fn):
    model.eval()
    y_pred = []
    y_target = []
    with torch.no_grad():
        for X,y,seq in dataloader:
            X,y = X.to(device) , y.to(device,dtype=torch.float)
            pred = model(X,seq)
            y_pred.extend(torch.sigmoid(pred).cpu().detach().numpy().tolist())
            y_target.extend(y.cpu().detach().numpy().tolist())
    y_preds = (np.array(y_pred)>0.5).astype(int)
    marco_f1= f1_score(y_target,y_preds,average='macro')
    print("marco f1 score : ",marco_f1)
    return marco_f1

In [214]:
val_data =load_data('val.json')
for i in val_data:
    sent_arr = set(preprocess(i['tweet']))
    if len(sent_arr) == 0:
        print(i)
        val_data.remove(i)
        continue
val_data = CustomDataset(val_data,preprocess,max_length,vocab2idx)
val_dataloader = DataLoader(val_data,batch_size=1)

# evalute(val_dataloader,model_2,loss_fn)

{'ID': '1397651228679675907', 'tweet': '      ', 'labels': {'none': [{'index': 0, 'start': 0, 'end': 1, 'terms': '@HHSGov'}]}}


In [215]:
epochs = 30
max_score = 0 
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------")
    train(train_loader,model_2,loss_fn,optimizer)
    score = evalute(val_dataloader,model_2,loss_fn)
    # print(f"max score in {t+1} epoch : ",score)
    if score > max_score:
        max_score =score
        path = './w_weight.pth'
        torch.save(model_2.state_dict(),path)
        print('save model')
print("best score: " ,max_score)

Epoch 1
-----------------------
loss: 0.695204 [    0/ 6955]


loss: 0.223062 [ 6400/ 6955]
marco f1 score :  0.10070200637160141
save model
Epoch 2
-----------------------
loss: 0.268645 [    0/ 6955]
loss: 0.205655 [ 6400/ 6955]
marco f1 score :  0.28560979677887705
save model
Epoch 3
-----------------------
loss: 0.181192 [    0/ 6955]
loss: 0.214052 [ 6400/ 6955]
marco f1 score :  0.39746167613913935
save model
Epoch 4
-----------------------
loss: 0.186528 [    0/ 6955]
loss: 0.188753 [ 6400/ 6955]
marco f1 score :  0.4580803159114042
save model
Epoch 5
-----------------------
loss: 0.166005 [    0/ 6955]
loss: 0.156546 [ 6400/ 6955]
marco f1 score :  0.4943530955103122
save model
Epoch 6
-----------------------
loss: 0.154780 [    0/ 6955]
loss: 0.137805 [ 6400/ 6955]
marco f1 score :  0.5232088155409657
save model
Epoch 7
-----------------------
loss: 0.132964 [    0/ 6955]
loss: 0.127536 [ 6400/ 6955]
marco f1 score :  0.5389555348351531
save model
Epoch 8
-----------------------
loss: 0.115263 [    0/ 6955]
loss: 0.116594 [ 6400/ 6955]
ma

In [216]:
class testDataset(Dataset):
    def __init__(self,data, tokenizer,max_length,words):
        self.data = data 
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.words = words

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index) :
        tweet = self.data[index]["tweet"]
        encode_text = self.encode(tweet,self.words)
        seq_len = np.count_nonzero(encode_text)
        return encode_text  , seq_len
    def encode(self,text,vocab2idx):
        token = self.tokenizer(text)
        encoded = np.zeros(self.max_length,dtype=int)
        enc1 = np.array([vocab2idx.get(word,vocab2idx["UNK"]) for word in token] )
        # print(enc1.shape)
        length = min(self.max_length,len(enc1))
        encoded[:length] = enc1[:length]
        return encoded 

In [217]:
test_data = load_data("test.json")
test_data = testDataset(test_data,preprocess,max_length,vocab2idx)
test_loader = DataLoader(test_data,batch_size=1)

In [218]:
path = './w_weight.pth'
model_2 = pretrain_weight_LSTM(vocab_size,embedding_dim,hidden_dim,pretrain_weights,bidirectional).to(device)
model_2.load_state_dict(torch.load(path))
model_2.eval()
y_pred = []
with torch.no_grad():
    for x,seq_len in test_loader:
        x = x.to(device)
        pred = model_2(x,seq_len)
        y_pred.extend(torch.sigmoid(pred).cpu().detach().numpy().tolist())

In [219]:
y_preds = (np.array(y_pred) > 0.5).astype(int)
y_preds

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [220]:
df = pd.DataFrame(y_preds, columns=tag_list)
df_reset = df.reset_index()
# print(df_reset.head())

In [221]:
data_rows = df_reset.to_dict(orient='records')

In [222]:
import csv
with open("submission.csv", 'w', newline='') as csvfile:
    fieldnames=["index","ineffective","unnecessary","pharma","rushed","side-effect","mandatory","country","ingredients","political","none","conspiracy","religious"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data_rows)